In [1]:
import numpy as np
import statsmodels.api as sm

In [2]:
Y = np.asarray([1,3,4,5,2,3,4]) + np.arange(1,8) - 5
x = np.arange(1,8)
weights = np.arange(1,8)
X = sm.add_constant(x)
wls_model = sm.WLS(Y, X, weights)
results = wls_model.fit()
results.params

array([-2.08333333,  1.0952381 ])

In [3]:
sm.WLS(Y / np.sqrt(weights), X, 1 / weights).fit().params

array([-3.05811757,  0.94131181])

In [4]:
sm.OLS(Y, X).fit().params

array([-2.85714286,  1.25      ])

In [5]:
import pymc3 as pm

In [6]:
with pm.Model() as m:
    intercept = pm.Normal('c', 0., 10.)
    beta = pm.Normal('b', 0., 5.)
    # Total variance
    sigma = pm.HalfNormal('s', 2.)
    # Scale by weights
    # The weights are presumed to be (proportional to) the 
    # inverse of the variance of the observations.
    tau = 1 / sigma**2 * (weights / weights.sum())
    pm.Normal('y', beta*x + intercept, tau=tau, observed=Y)
    trace = pm.sample(return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [s, b, c]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 4 seconds.
There were 97 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6340178266612129, but should be close to 0.8. Try to increase the number of tuning steps.
There were 36 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


In [7]:
pm.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
c,-1.971,1.887,-5.536,1.518,0.107,0.075,313.0,313.0,314.0,578.0,1.01
b,1.076,0.359,0.429,1.738,0.021,0.015,307.0,307.0,304.0,464.0,1.00
s,0.618,0.257,0.313,1.047,0.016,0.011,272.0,272.0,150.0,123.0,1.00


In [16]:
with pm.Model() as m2:
    intercept = pm.Normal('c', 0., 10.)
    beta = pm.Normal('b', 0., 5.)
    pm.Normal('y', beta*x + intercept, tau=weights, observed=Y)
    map_result = pm.find_MAP()
map_result

{'c': array(-2.07381709), 'b': array(1.09348298)}